In [ ]:
%%capture
# !pip install ffmpeg-python
!pip install -q typing-extensions==4.5.0 kaleido accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
!pip install torchaudio

from transformers import WhisperForConditionalGeneration, WhisperProcessor,WhisperTokenizer,pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model= WhisperForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Colab Notebooks/model_test/Model_from_huggingface')

In [ ]:
processor = WhisperProcessor.from_pretrained("/content/drive/MyDrive/Colab Notebooks/model_test/Model_from_huggingface")

In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/model_test/Model_from_huggingface")  # Replace with your model identifier

In [ ]:
pipe = pipeline('automatic-speech-recognition', tokenizer=tokenizer, model=model,feature_extractor=processor)

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read, write as wav_write
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio(output_path='audio.wav'):
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))
  wav_write(output_path, sr, audio)

  return audio, sr, output_path

In [ ]:
audio, sr,audio_path = get_audio()

In [ ]:
import torchaudio

def load_audio(audio_path, target_sampling_rate=16000):
    waveform, sample_rate = torchaudio.load(audio_path)
    if waveform.dim() > 1:
        waveform = waveform[0]

    if sample_rate != target_sampling_rate:
        waveform = torchaudio.transforms.Resample(sample_rate, target_sampling_rate)(waveform)
        sample_rate = target_sampling_rate

    audio_info = {
        "array": waveform,
        "sampling_rate": sample_rate,
    }

    return audio_info

In [ ]:
audio_data = load_audio(audio_path)
inputs = processor(audio_data['array'], return_tensors="pt")
input_features = inputs.input_features
generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:472: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [ ]:
transcription

'move to XY location'